In [1]:
from cloud_spider import NWSpider

In [2]:
url = 'https://www.networkworld.com/cloud-computing/'
spider = NWSpider(url)
spider.extract()

In [23]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime as dt
from sqlalchemy import create_engine
import os
from urllib.request import Request, urlopen


In [2]:
class NWSpider():
    def __init__(self, url):
        conn_url = os.environ.get('POSTGRES_CONNECTION_STRING')
        self.engine = create_engine(f'{conn_url}cloudnewsdb')
        self.data = []
        self.url = url

        source = requests.get(url).text
        self.soup = BeautifulSoup(source, 'lxml')

    def extract(self):
        for a in self.soup.find_all('a', attrs={'class': 'card'}):
            date = a.find_next('div', attrs={'class': 'card__info--light'}).find_next('span').text
            if 'dec' in date.lower():
                self.data.append({
                    'title': a.find_next('h4').text,
                    'url': a['href'],
                    'date': date,
                    'source': 'Network World',
                    'query_date': dt.now()
                })
        df = pd.DataFrame(self.data)
        df.to_sql('raw', self.engine, if_exists='replace')

In [8]:
dt.strftime(dt.today(), format='%b').lower()

'jan'

In [24]:
url = 'https://www.ciodive.com/topic/cloud/'
source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')

In [13]:
data = []
for li in soup.select('li.row.feed__item'):
    a = li.find_next('h3', attrs={'class': 'feed__title'}).find_next('a')
    url = a['href']
    title = a.text.strip()
    try:
        date = li.find_all('span', attrs={'class':'secondary-label'})[1].text
    except:
        pass
    data.append({
    'title':title,
    'url': url,
    'date': date,
    'source': 'Ciodive',
    'query_date': dt.now()
    })

In [25]:
url = 'https://www.cloudcomputing-news.net/categories/cloud-computing-companies/'
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
page = urlopen(req).read()
soup = BeautifulSoup(page, 'lxml')

In [30]:
data = []
for article in soup.find_all('article', attrs={'class':'archive-post'}):
    link =  article.find_next('header').find_next('a')
    byline = article.find_next('div', attrs={'class':'byline'})
    data.append({
    'title':a.text.strip(),
    'url': a['href'],
    'date': byline.text.split('|')[0].strip(),
    'source': 'Cloudtech',
    'query_date': dt.now()
    })
data

[{'title': 'FTC extends cloud competition scrutiny to generative AI',
  'url': '/news/FTC-cloud-competition-risk-hearing-generative-AI-Lina-Khan/700232/',
  'date': '10 January 2024',
  'source': 'Cloudtech',
  'query_date': datetime.datetime(2024, 1, 12, 14, 16, 23, 824811)},
 {'title': 'FTC extends cloud competition scrutiny to generative AI',
  'url': '/news/FTC-cloud-competition-risk-hearing-generative-AI-Lina-Khan/700232/',
  'date': '9 January 2024',
  'source': 'Cloudtech',
  'query_date': datetime.datetime(2024, 1, 12, 14, 16, 23, 825810)},
 {'title': 'FTC extends cloud competition scrutiny to generative AI',
  'url': '/news/FTC-cloud-competition-risk-hearing-generative-AI-Lina-Khan/700232/',
  'date': '3 January 2024',
  'source': 'Cloudtech',
  'query_date': datetime.datetime(2024, 1, 12, 14, 16, 23, 825810)},
 {'title': 'FTC extends cloud competition scrutiny to generative AI',
  'url': '/news/FTC-cloud-competition-risk-hearing-generative-AI-Lina-Khan/700232/',
  'date': '2 